<a href="https://colab.research.google.com/github/ummd/ummd.github.io/blob/master/Teaching/RSM3053/Lecture3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
install.packages('fabricatr')
install.packages('DeclareDesign')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Formula’, ‘Rcpp’, ‘RcppEigen’, ‘randomizr’, ‘estimatr’




In [4]:
library(fabricatr)

voters <- fabricate(
  N = 1000,
  group_id = rep(1:10, 100),
  ideology = draw_normal_icc(mean = 0, N = N, clusters = group_id, ICC = 0.7),
  ideological_label = draw_ordered(
    x = ideology,
    break_labels = c(
      "Very Conservative", "Conservative",
      "Liberal", "Very Liberal"
    )
  ),
  income = exp(rlnorm(n = N, meanlog = 2.4 - (ideology * 0.1), sdlog = 0.12)),
  Q1_immigration = draw_likert(x = ideology, min = -5, max = 5, bins = 7),
  Q2_defence = draw_likert(x = ideology + 0.5, min = -5, max = 5, bins = 7),
  treatment = draw_binary(0.5, N = N),
  proposition_vote = draw_binary(latent = ideology + 1.2 * treatment, link = "probit")
)
head(voters)

,ID,group_id,ideology,ideological_label,income,Q1_immigration,Q2_defence,treatment,proposition_vote
,<chr>,<int>,<dbl>,<ord>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,0001,1,-0.7714024,Conservative,212314.813,3,4,0,0
2,0002,2,-3.8702572,Very Conservative,22100314.577,1,2,1,0
3,0003,3,-1.9843458,Very Conservative,649130.465,3,3,0,0
4,0004,4,-0.1569052,Conservative,241317.615,4,4,0,0
5,0005,5,2.7133168,Very Liberal,3262.371,6,6,1,1
6,0006,6,-0.4318626,Conservative,104985.836,4,4,1,1


In [19]:
panel <- fabricate(
  subjects = add_level(N = 150, subj_fe = runif(N, 1, 10)),
  trials = add_level(N = 25, trial_shock = runif(N, 1, 10), nest = FALSE),
  observations = cross_levels(
    by = join_using(trials,subjects),
    outcome_it = subj_fe + trial_shock + rnorm(N, 0, 2)
  )
)

In [20]:
panel <- fabricate(
  subjects = add_level(N = 150, subj_fe = runif(N, 1, 10)),
  trials = add_level(N = 25, trial_shock = runif(N, 1, 10), nest = FALSE),
  observations = cross_levels(
    by = join_using(trials,subjects),
    outcome_it = draw_likert(x = subj_fe + trial_shock + rnorm(N, 0, 2), min = 0, max = 20, bins = 7)
  )
)

In [40]:
library(DeclareDesign)

design <-
  declare_model(
    N = 100,
    potential_outcomes(Y ~ rbinom(N, size = 1, prob = 0.5 + 0.2 * Z))
  ) +
  declare_inquiry(ATE = 0.2) +
  declare_assignment(Z = complete_ra(N, m = 50)) +
  declare_measurement(Y = reveal_outcomes(Y ~ Z)) +
  declare_estimator(Y ~ Z, .method = lm_robust, inquiry = "ATE")

diagnosands <-
  declare_diagnosands(bias = mean(estimate - estimand),
                      power = mean(p.value <= 0.05))

diagnosis <- diagnose_design(design, diagnosands = diagnosands)
diagnosis


Research design diagnosis based on 500 simulations. Diagnosis completed in 6 secs. Diagnosand estimates with bootstrapped standard errors in parentheses (100 replicates).

 Design Inquiry Estimator Outcome Term N Sims   Bias  Power
 design     ATE estimator       Y    Z    500  -0.00   0.51
                                              (0.00) (0.02)

In [41]:
designs <- redesign(design, N = c(100, 200, 300, 500))
diagnose_design(designs)


Research design diagnosis based on 500 simulations. Diagnosis completed in 26 secs. Diagnosand estimates with bootstrapped standard errors in parentheses (100 replicates).

   Design   N Inquiry Estimator Outcome Term N Sims Mean Estimand Mean Estimate
 design_1 100     ATE estimator       Y    Z    500          0.20          0.20
                                                           (0.00)        (0.00)
 design_2 200     ATE estimator       Y    Z    500          0.20          0.20
                                                           (0.00)        (0.00)
 design_3 300     ATE estimator       Y    Z    500          0.20          0.20
                                                           (0.00)        (0.00)
 design_4 500     ATE estimator       Y    Z    500          0.20          0.21
                                                           (0.00)        (0.00)
   Bias SD Estimate   RMSE  Power Coverage
  -0.00        0.09   0.09   0.54     0.96
 (0.00)      (0.00) 

In [44]:
library(DeclareDesign)

design <-
  declare_model(
    N = 100,
    potential_outcomes(Y ~ draw_likert(x = Z + 10 + rnorm(N, 0, 2), min = 0, max = 20, bins = 7))
  ) +
  declare_inquiry(ATE = 1) +
  declare_assignment(Z = complete_ra(N, m = 50)) +
  declare_measurement(Y = reveal_outcomes(Y ~ Z)) +
  declare_estimator(Y ~ Z, .method = lm_robust, inquiry = "ATE")

diagnosands <-
  declare_diagnosands(bias = mean(estimate - estimand),
                      power = mean(p.value <= 0.05))

diagnosis <- diagnose_design(design, diagnosands = diagnosands)
diagnosis


Research design diagnosis based on 500 simulations. Diagnosis completed in 7 secs. Diagnosand estimates with bootstrapped standard errors in parentheses (100 replicates).

 Design Inquiry Estimator Outcome Term N Sims   Bias  Power
 design     ATE estimator       Y    Z    500  -0.65   0.61
                                              (0.01) (0.02)